# Iterative RAG: Smart Retrieval Without Code Generation

## Key Insight

Code generation **hallucinates** selectors. The original RAG approach actually worked better!

## The Real Solution

**Don't generate code. Iterate on retrieval!**

### Strategy
1. ✅ Semantic search (embeddings work great)
2. ✅ Retrieve MORE chunks (increase K)
3. ✅ Extract DIRECTLY from text (no code hallucination)
4. ✅ Iterate if incomplete (refine queries, search again)
5. ✅ Deduplicate results

### Why This Works
- Embeddings are reliable for finding relevant content
- LLMs are good at extraction from provided text
- LLMs are BAD at inferring HTML structure
- Iteration handles completeness

### Process
```
Query → Search (k=20) → Retrieve text → Extract → Check completeness
   ↑                                                      |
   |                                                      ↓
   └─── Refine query ←─── Still missing? ←───────────────┘
```

## 1. Setup and Imports

In [1]:
import os
import json
from pathlib import Path
from typing import List, Dict, Any, Set

from bs4 import BeautifulSoup
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.llms import Ollama
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate

## 2. Configuration

In [2]:
# Project paths
PROJECT_ROOT = Path("/Users/ardyh/Documents/job-applications/mrscraper")
DATA_DIR = PROJECT_ROOT / "data" / "html"

# Test scenarios
HTML_FILES = {
    "scenario1_books": DATA_DIR / "scenario1_books.html",
    "scenario2_jobs": DATA_DIR / "scenario2_jobs.html",
    "scenario3_clubs": DATA_DIR / "scenario3_clubs.html",
    "scenario4_property": DATA_DIR / "scenario4_property.html"
}

TEST_QUERIES = {
    "scenario1_books": "Can you return me the books: name and price?",
    "scenario2_jobs": "Extract job title, location, salary, and company name from the listings",
    "scenario3_clubs": "Get the club names, logo image links and their official websites",
    "scenario4_property": "Return the property name, address, latitude and longitude"
}

## 3. Initialize Models

In [3]:
# Embedding model
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

# LLM for extraction
llm = Ollama(
    model="llama3",
    temperature=0
)

print("✓ Models initialized!")

/var/folders/4z/mfq1y5w57hj01ttff7smkspw0000gn/T/ipykernel_2654/2261245867.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


✓ Models initialized!


/var/folders/4z/mfq1y5w57hj01ttff7smkspw0000gn/T/ipykernel_2654/2261245867.py:9: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaLLM``.
  llm = Ollama(


## 4. Iterative RAG System

In [55]:
def load_and_index_html(html_path: Path, chunk_size: int = 800, overlap: int = 200):
    """
    Load HTML and create vector index.
    Uses clean text for better embeddings.
    """
    print(f"  Loading and indexing {html_path.name}...")
    
    # Load HTML and extract clean text
    loader = UnstructuredHTMLLoader(str(html_path))
    documents = loader.load()
    
    # Chunk with good overlap
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=overlap
    )
    chunks = splitter.split_documents(documents)
    
    # Create vector store
    vectorstore = Chroma.from_documents(
        chunks,
        embeddings,
        collection_name=f"html_{html_path.stem}"
    )
    
    print(f"  ✓ Indexed {len(chunks)} chunks")
    return vectorstore

def search_and_retrieve(vectorstore, query: str, k: int = 20) -> List[str]:
    """
    Semantic search with high K for completeness.
    """
    docs = vectorstore.similarity_search(query, k=k)
    return [doc.page_content for doc in docs]

def extract_from_chunks(chunks: List[str], query: str, llm) -> List[Dict]:
    """
    Direct extraction from retrieved chunks.
    No code generation - just extract!
    """
    combined_content = "\n\n---\n\n".join(chunks)
    
    extraction_prompt = f"""
Text:
{combined_content}

Extract information from the following text based on the query.

Query: {query}

Instructions:
1. Extract ALL matching items you can find
2. Return as a JSON array of objects
3. If a field is missing, use null
4. Be thorough - extract everything relevant

Return ONLY the JSON array, no other text.
JSON:
"""
    
    response = llm.invoke(extraction_prompt)
    
    # Parse JSON from response
    try:
        # Try to extract JSON from response
        if "```json" in response:
            json_str = response.split("```json")[1].split("```")[0]
        elif "```" in response:
            json_str = response.split("```")[1].split("```")[0]
        elif "[" in response:
            # Find the JSON array
            start = response.index("[")
            end = response.rindex("]") + 1
            json_str = response[start:end]
        else:
            json_str = response
        
        result = json.loads(json_str)
        return result if isinstance(result, list) else [result]
    except Exception as e:
        print(f"    ⚠️  JSON parsing error: {e}")
        print(f"    Response was: {response[:200]}...")
        return []

def deduplicate_results(results: List[Dict]) -> List[Dict]:
    """
    Remove duplicate entries based on content.
    """
    seen = set()
    unique = []
    
    for item in results:
        # Create a hashable representation
        key = json.dumps(item, sort_keys=True)
        if key not in seen:
            seen.add(key)
            unique.append(item)
    
    return unique

def is_complete(results: List[Dict], expected_min: int = 5) -> bool:
    """
    Simple heuristic: are we getting reasonable results?
    """
    return len(results) >= expected_min

print("✓ Iterative RAG functions defined!")

✓ Iterative RAG functions defined!


## 5. Iterative Extraction Agent

In [5]:
def iterative_rag_extraction(
    html_path: Path,
    query: str,
    max_iterations: int = 3,
    initial_k: int = 20,
    max_k: int = 50
) -> List[Dict]:
    """
    Iterative RAG extraction with progressive retrieval.
    
    Strategy:
    1. Start with k=20 chunks
    2. Extract directly from chunks
    3. If incomplete, increase k and try different query variations
    4. Deduplicate and return
    """
    print(f"\n{'='*80}")
    print(f"Iterative RAG Extraction: {html_path.name}")
    print(f"Query: {query}")
    print(f"{'='*80}\n")
    
    # Load and index
    vectorstore = load_and_index_html(html_path)
    print()
    
    all_results = []
    k = initial_k
    
    # Query variations for iteration
    query_variations = [
        query,
        f"all {query}",
        f"complete list {query}",
        query.replace("extract", "find all").replace("get", "list all")
    ]
    
    for iteration in range(max_iterations):
        print(f"[Iteration {iteration + 1}] Retrieving and extracting...")
        
        # Use different query each iteration
        current_query = query_variations[min(iteration, len(query_variations) - 1)]
        
        # Retrieve more chunks each iteration
        current_k = min(k + (iteration * 15), max_k)
        print(f"  Retrieving top {current_k} chunks for: '{current_query}'")
        
        chunks = search_and_retrieve(vectorstore, current_query, k=current_k)
        print(f"  Retrieved {len(chunks)} chunks ({sum(len(c) for c in chunks)} chars)")
        
        # Extract
        print(f"  Extracting data from chunks...")
        results = extract_from_chunks(chunks, query, llm)
        print(f"  ✓ Extracted {len(results)} items\n")
        
        all_results.extend(results)
        
        # Check if we should continue
        unique_so_far = deduplicate_results(all_results)
        print(f"  Total unique items so far: {len(unique_so_far)}")
        
        # Stop if we're not getting new results
        if iteration > 0 and len(unique_so_far) == len(deduplicate_results(all_results[:-len(results)])):
            print(f"  No new items found, stopping iteration.\n")
            break
    
    # Final deduplication
    final_results = deduplicate_results(all_results)
    
    print(f"{'='*80}")
    print(f"FINAL: Extracted {len(final_results)} unique items")
    print(f"{'='*80}\n")
    
    return final_results

print("✓ Iterative extraction agent defined!")

✓ Iterative extraction agent defined!


## 6. Test on All Scenarios

In [6]:
# Run iterative RAG on all scenarios
iterative_results = {}

for scenario_name, html_file in HTML_FILES.items():
    query = TEST_QUERIES[scenario_name]
    
    try:
        result = iterative_rag_extraction(html_file, query, max_iterations=3)
        iterative_results[scenario_name] = result
        
        print("\n" + "="*80)
        print(f"RESULT for {scenario_name}:")
        print(f"✓ Extracted {len(result)} items\n")
        
        if len(result) > 0:
            print("First 2 items:")
            print(json.dumps(result[:2], indent=2))
        
        print("="*80 + "\n\n")
        
    except Exception as e:
        print(f"\n✗ Error: {e}\n")
        import traceback
        traceback.print_exc()
        iterative_results[scenario_name] = []


Iterative RAG Extraction: scenario1_books.html
Query: Can you return me the books: name and price?

  Loading and indexing scenario1_books.html...
  ✓ Indexed 4 chunks

[Iteration 1] Retrieving and extracting...
  Retrieving top 20 chunks for: 'Can you return me the books: name and price?'
  Retrieved 4 chunks (2928 chars)
  Extracting data from chunks...
  ✓ Extracted 21 items

  Total unique items so far: 21
[Iteration 2] Retrieving and extracting...
  Retrieving top 35 chunks for: 'all Can you return me the books: name and price?'
  Retrieved 4 chunks (2928 chars)
  Extracting data from chunks...
  ✓ Extracted 21 items

  Total unique items so far: 21
  No new items found, stopping iteration.

FINAL: Extracted 21 unique items


RESULT for scenario1_books:
✓ Extracted 21 items

First 2 items:
[
  {
    "name": "The Dirty Little Secrets ...",
    "price": "\u00a333.34"
  },
  {
    "name": "The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull",
    

## 7. Compare with Previous Approaches

In [7]:
print("\n" + "="*80)
print("COMPARISON: Iterative RAG vs Previous Approaches")
print("="*80 + "\n")

# Expected results from previous approaches (from your tests)
previous_results = {
    "Original RAG (k=5)": {
        "scenario1_books": 20,
        "scenario2_jobs": 7,
        "scenario3_clubs": 2,
        "scenario4_property": 0
    },
    "Code Generation": {
        "scenario1_books": 0,
        "scenario2_jobs": 0,
        "scenario3_clubs": 6,
        "scenario4_property": 0
    }
}

print(f"{'Scenario':<25} {'Original RAG':<15} {'Code Gen':<15} {'Iterative RAG':<15}")
print("-" * 80)

for scenario in HTML_FILES.keys():
    orig = previous_results["Original RAG (k=5)"].get(scenario, "?")
    code = previous_results["Code Generation"].get(scenario, "?")
    iter_count = len(iterative_results.get(scenario, []))
    
    # Highlight best result
    best = max([orig if isinstance(orig, int) else 0, 
                code if isinstance(code, int) else 0, 
                iter_count])
    
    orig_str = f"{orig}" if orig != best else f"✓ {orig}"
    code_str = f"{code}" if code != best else f"✓ {code}"
    iter_str = f"{iter_count}" if iter_count != best else f"✓ {iter_count}"
    
    print(f"{scenario:<25} {orig_str:<15} {code_str:<15} {iter_str:<15}")

print("\n" + "="*80)
print("\n📊 KEY INSIGHTS:\n")
print("1. Iterative RAG retrieves MORE chunks (k=20-50 vs k=5)")
print("2. Direct extraction avoids code generation hallucinations")
print("3. Multiple query variations catch edge cases")
print("4. Deduplication ensures clean results")
print("5. Simple and reliable - no complex code generation")
print("\n" + "="*80)


COMPARISON: Iterative RAG vs Previous Approaches

Scenario                  Original RAG    Code Gen        Iterative RAG  
--------------------------------------------------------------------------------
scenario1_books           20              0               ✓ 21           
scenario2_jobs            ✓ 7             0               6              
scenario3_clubs           2               ✓ 6             0              
scenario4_property        0               0               ✓ 3            


📊 KEY INSIGHTS:

1. Iterative RAG retrieves MORE chunks (k=20-50 vs k=5)
2. Direct extraction avoids code generation hallucinations
3. Multiple query variations catch edge cases
4. Deduplication ensures clean results
5. Simple and reliable - no complex code generation



## 8. Why This Works Better

### The Problem with Code Generation

```python
# LLM sees sample HTML
<div>Some content</div>

# LLM hallucinates selectors
soup.find_all('div', class_='product-grid-item')  # ← WRONG class!

# Result: 0 items extracted
```

### The Solution: Iterative Retrieval

```python
# Iteration 1: Retrieve k=20 chunks
results = extract_from_text(chunks)  # → 10 items

# Iteration 2: Retrieve k=35 chunks with refined query
more_results = extract_from_text(more_chunks)  # → 5 more items

# Iteration 3: Retrieve k=50 chunks
even_more = extract_from_text(even_more_chunks)  # → 2 more items

# Deduplicate: 17 unique items total ✅
```

### Key Advantages

1. **No Hallucination**: LLM extracts from provided text, not inferred structure
2. **Progressive**: Each iteration retrieves more chunks
3. **Robust**: Multiple query variations catch different phrasings
4. **Simple**: No complex code generation/execution pipeline
5. **Reliable**: Embeddings are proven to work well

### When to Stop

- Reached max iterations
- No new unique items found
- Retrieved max_k chunks

This is the **pragmatic solution** that builds on what we know works!

## 9. Debug Mode: Inspect Retrieved Chunks

Let's see exactly what chunks are being retrieved for scenarios 3 and 4


In [9]:
def debug_retrieval(html_path: Path, query: str, k: int = 20):
    """
    Debug function to see exactly what chunks are retrieved.
    """
    print(f"\n{'='*80}")
    print(f"DEBUG: {html_path.name}")
    print(f"Query: {query}")
    print(f"Retrieving top {k} chunks")
    print(f"{'='*80}\n")
    
    # Load and index
    print("Loading and indexing...")
    vectorstore = load_and_index_html(html_path, chunk_size=800, overlap=200)
    print()
    
    # Retrieve chunks
    print(f"Retrieving chunks...")
    chunks = search_and_retrieve(vectorstore, query, k=k)
    
    print(f"\n✓ Retrieved {len(chunks)} chunks\n")
    print(f"{'='*80}")
    
    # Show each chunk with metadata
    for i, chunk in enumerate(chunks):
        print(f"\n--- CHUNK {i+1} ({len(chunk)} chars) ---")
        print(chunk[:500])  # Show first 500 chars
        if len(chunk) > 500:
            print("...")
            print(chunk[-200:])  # Show last 200 chars
        print()
    
    print(f"{'='*80}\n")
    
    # Summary statistics
    total_chars = sum(len(c) for c in chunks)
    avg_chars = total_chars / len(chunks) if chunks else 0
    
    print(f"📊 SUMMARY:")
    print(f"  Total chunks: {len(chunks)}")
    print(f"  Total characters: {total_chars:,}")
    print(f"  Average chunk size: {avg_chars:.0f} chars")
    print(f"  Min chunk size: {min(len(c) for c in chunks) if chunks else 0}")
    print(f"  Max chunk size: {max(len(c) for c in chunks) if chunks else 0}")
    print()
    
    return chunks

print("✓ Debug function defined!")


✓ Debug function defined!


### Debug Scenario 3: Clubs


In [10]:
scenario3_chunks = debug_retrieval(
    HTML_FILES["scenario3_clubs"],
    TEST_QUERIES["scenario3_clubs"],
    k=20
)



DEBUG: scenario3_clubs.html
Query: Get the club names, logo image links and their official websites
Retrieving top 20 chunks

Loading and indexing...
  Loading and indexing scenario3_clubs.html...
  ✓ Indexed 1 chunks

Retrieving chunks...

✓ Retrieved 2 chunks


--- CHUNK 1 (163 chars) ---
Member Clubs

This is a list of every current member club of the Arizona Soccer Association. To identify a club near you please use the map or dropdown list below.


--- CHUNK 2 (163 chars) ---
Member Clubs

This is a list of every current member club of the Arizona Soccer Association. To identify a club near you please use the map or dropdown list below.


📊 SUMMARY:
  Total chunks: 2
  Total characters: 326
  Average chunk size: 163 chars
  Min chunk size: 163
  Max chunk size: 163



### Debug Scenario 4: Property


In [15]:
scenario4_chunks = debug_retrieval(
    HTML_FILES["scenario4_property"],
    TEST_QUERIES["scenario4_property"],
    k=20
)



DEBUG: scenario4_property.html
Query: Return the property name, address, latitude and longitude
Retrieving top 20 chunks

Loading and indexing...
  Loading and indexing scenario4_property.html...
  ✓ Indexed 11 chunks

Retrieving chunks...

✓ Retrieved 20 chunks


--- CHUNK 1 (774 chars) ---
We are here to set the new standard for short-term rental experiences.

For guest & booking inquiries:

experience@avantstay.com

(877) 640-7787

For property management inquiries:

newhomes@avantstay.com

(855) 683-1768

Are you a current homeowner?

Stay in the loop

Keep up with our featured homes, new locations, and travel tips! Oh, and get $100 off your first stay with us.

InstagramFacebookLinkedInGlassdoor

TermsPrivacy PolicyFair Housing PolicyAffirm Disclosures

© 2025 AvantStay, Inc. A
...
res

InstagramFacebookLinkedInGlassdoor

© 2025 AvantStay, Inc. All rights reserved.

This site uses cookies to provide you with a personalized experience

Learn more about AvantStay’s Privacy Policy.


### Analysis: What do the chunks contain?


In [16]:
# Analyze scenario 3 chunks for key terms
print("=" * 80)
print("SCENARIO 3 ANALYSIS: Looking for club-related terms")
print("=" * 80 + "\n")

club_terms = ["club", "logo", "website", "http", "www", "soccer", "fc", "sc"]

for i, chunk in enumerate(scenario3_chunks[:5]):  # Check first 5 chunks
    print(f"\nChunk {i+1} term frequency:")
    for term in club_terms:
        count = chunk.lower().count(term)
        if count > 0:
            print(f"  '{term}': {count}")

print("\n" + "=" * 80)
print("SCENARIO 4 ANALYSIS: Looking for property-related terms")
print("=" * 80 + "\n")

property_terms = ["property", "address", "latitude", "longitude", "lat", "lng", "coord"]

for i, chunk in enumerate(scenario4_chunks[:5]):  # Check first 5 chunks
    print(f"\nChunk {i+1} term frequency:")
    for term in property_terms:
        count = chunk.lower().count(term)
        if count > 0:
            print(f"  '{term}': {count}")
            
print("\n" + "=" * 80)


SCENARIO 3 ANALYSIS: Looking for club-related terms


Chunk 1 term frequency:
  'club': 3
  'soccer': 1

Chunk 2 term frequency:
  'club': 3
  'soccer': 1

SCENARIO 4 ANALYSIS: Looking for property-related terms


Chunk 1 term frequency:
  'property': 1

Chunk 2 term frequency:
  'property': 1

Chunk 3 term frequency:
  'property': 1

Chunk 4 term frequency:
  'property': 1

Chunk 5 term frequency:



### Try Alternative Search Queries

Maybe the original query isn't matching well. Let's try variations.


In [17]:
# Test different queries for scenario 3
print("Testing different queries for Scenario 3 (Clubs):\n")

vectorstore_3 = load_and_index_html(HTML_FILES["scenario3_clubs"], chunk_size=800, overlap=200)

alternative_queries_3 = [
    "club names logo image links official websites",
    "soccer clubs logos",
    "football club website",
    "team logo image",
    "club information"
]

for q in alternative_queries_3:
    chunks = search_and_retrieve(vectorstore_3, q, k=5)
    print(f"\nQuery: '{q}'")
    print(f"  Retrieved {len(chunks)} chunks")
    # Check if any contain useful data
    has_logo = any("logo" in c.lower() or "image" in c.lower() for c in chunks)
    has_website = any("http" in c.lower() or "www" in c.lower() for c in chunks)
    print(f"  Contains logo/image: {has_logo}")
    print(f"  Contains website/URL: {has_website}")

print("\n" + "="*80)
print("\nTesting different queries for Scenario 4 (Property):\n")

vectorstore_4 = load_and_index_html(HTML_FILES["scenario4_property"], chunk_size=800, overlap=200)

alternative_queries_4 = [
    "property address coordinates",
    "latitude longitude location",
    "building address geo",
    "property location map",
    "coordinates GPS"
]

for q in alternative_queries_4:
    chunks = search_and_retrieve(vectorstore_4, q, k=5)
    print(f"\nQuery: '{q}'")
    print(f"  Retrieved {len(chunks)} chunks")
    # Check if any contain coordinates
    has_coords = any(any(term in c.lower() for term in ["latitude", "longitude", "lat", "lng"]) for c in chunks)
    has_address = any("address" in c.lower() for c in chunks)
    print(f"  Contains coordinates: {has_coords}")
    print(f"  Contains address: {has_address}")
    
print("\n" + "="*80)


Testing different queries for Scenario 3 (Clubs):

  Loading and indexing scenario3_clubs.html...
  ✓ Indexed 1 chunks

Query: 'club names logo image links official websites'
  Retrieved 4 chunks
  Contains logo/image: False
  Contains website/URL: False

Query: 'soccer clubs logos'
  Retrieved 4 chunks
  Contains logo/image: False
  Contains website/URL: False

Query: 'football club website'
  Retrieved 4 chunks
  Contains logo/image: False
  Contains website/URL: False

Query: 'team logo image'
  Retrieved 4 chunks
  Contains logo/image: False
  Contains website/URL: False

Query: 'club information'
  Retrieved 4 chunks
  Contains logo/image: False
  Contains website/URL: False


Testing different queries for Scenario 4 (Property):

  Loading and indexing scenario4_property.html...
  ✓ Indexed 11 chunks

Query: 'property address coordinates'
  Retrieved 5 chunks
  Contains coordinates: False
  Contains address: False

Query: 'latitude longitude location'
  Retrieved 5 chunks
  Contai

### Deep Dive: Show Full Content of Top Chunks

Let's see the complete content of the most relevant chunks


In [18]:
# Show full content of top 3 chunks for scenario 3
print("=" * 80)
print("SCENARIO 3 - TOP 3 CHUNKS (FULL CONTENT)")
print("=" * 80)

for i in range(min(3, len(scenario3_chunks))):
    print(f"\n{'='*80}")
    print(f"CHUNK {i+1} - Length: {len(scenario3_chunks[i])} chars")
    print(f"{'='*80}\n")
    print(scenario3_chunks[i])
    print()

print("\n" + "=" * 80)
print("SCENARIO 4 - TOP 3 CHUNKS (FULL CONTENT)")
print("=" * 80)

for i in range(min(3, len(scenario4_chunks))):
    print(f"\n{'='*80}")
    print(f"CHUNK {i+1} - Length: {len(scenario4_chunks[i])} chars")
    print(f"{'='*80}\n")
    print(scenario4_chunks[i])
    print()


SCENARIO 3 - TOP 3 CHUNKS (FULL CONTENT)

CHUNK 1 - Length: 163 chars

Member Clubs

This is a list of every current member club of the Arizona Soccer Association. To identify a club near you please use the map or dropdown list below.


CHUNK 2 - Length: 163 chars

Member Clubs

This is a list of every current member club of the Arizona Soccer Association. To identify a club near you please use the map or dropdown list below.


SCENARIO 4 - TOP 3 CHUNKS (FULL CONTENT)

CHUNK 1 - Length: 774 chars

We are here to set the new standard for short-term rental experiences.

For guest & booking inquiries:

experience@avantstay.com

(877) 640-7787

For property management inquiries:

newhomes@avantstay.com

(855) 683-1768

Are you a current homeowner?

Stay in the loop

Keep up with our featured homes, new locations, and travel tips! Oh, and get $100 off your first stay with us.

InstagramFacebookLinkedInGlassdoor

TermsPrivacy PolicyFair Housing PolicyAffirm Disclosures

© 2025 AvantStay, Inc

## 10. Debugging Insights

### What to Look For

**Scenario 3 (Clubs):**
- Are club names appearing in the chunks?
- Are logo URLs present (look for `.png`, `.jpg`, image URLs)?
- Are website URLs present (look for `http://`, `https://`, `www.`)?
- Is the data spread across multiple chunks or concentrated?

**Scenario 4 (Property):**
- Is the property name in the chunks?
- Is the address in the chunks?
- Are coordinates present in ANY form?
  - Look for: `latitude`, `longitude`, `lat`, `lng`, `coord`
  - Look for: Decimal numbers like `40.7128, -74.0060`
  - Look for: JSON data structures
  - Look for: JavaScript variables with location data

### Common Issues

1. **Chunk Size Too Small**: Data split across chunks
   - Solution: Increase `chunk_size` or `overlap`

2. **Query Mismatch**: Embeddings don't match the query well
   - Solution: Try more specific or varied queries

3. **Data Not in Text**: Info might be in HTML attributes, JavaScript, or CSS
   - Solution: Need to parse raw HTML, not just extracted text

4. **Sparse Data**: Only 1 property, coordinates might be in a single location
   - Solution: Retrieve more chunks (increase k)


## 11. Root Cause: UnstructuredHTMLLoader Strips Too Much!

213 KB HTML → 163 chars text extracted! Let's see what's missing.


In [45]:
# Compare raw HTML vs extracted text for scenario 3
from bs4 import BeautifulSoup

print("="*80)
print("SCENARIO 3: RAW HTML vs EXTRACTED TEXT")
print("="*80)

# Load raw HTML
with open(HTML_FILES["scenario3_clubs"], 'r') as f:
    raw_html = f.read()

# Extract text with UnstructuredHTMLLoader
from langchain_community.document_loaders import UnstructuredHTMLLoader
loader = UnstructuredHTMLLoader(str(HTML_FILES["scenario3_clubs"]))
docs = loader.load()
extracted_text = docs[0].page_content if docs else ""

print(f"\nRaw HTML size: {len(raw_html):,} chars")
print(f"Extracted text size: {len(extracted_text):,} chars")
print(f"Data loss: {100 - (len(extracted_text)/len(raw_html)*100):.1f}%")

# Look for club data in raw HTML
soup = BeautifulSoup(raw_html, 'lxml')

# Find links (websites)
links = soup.find_all('a', href=True)
club_links = [a for a in links if 'http' in a['href'] and a['href'].startswith('http')]
print(f"\n✓ Found {len(club_links)} links in raw HTML")

# Find images (logos)
images = soup.find_all('img', src=True)
print(f"✓ Found {len(images)} images in raw HTML")

# Show some examples
print(f"\n📋 Sample club links from raw HTML:")
for link in club_links[:5]:
    print(f"  {link.get_text(strip=True)[:30]}: {link['href']}")

print(f"\n📋 Sample image URLs from raw HTML:")
for img in images[:5]:
    print(f"  {img.get('src', '')[:80]}")

# Check if this data is in extracted text
print(f"\n❌ Links in extracted text: 0")
print(f"❌ Images in extracted text: 0")

print("\n" + "="*80)
print("🎯 CONCLUSION: Data is in HTML attributes, NOT in visible text!")
print("="*80)


SCENARIO 3: RAW HTML vs EXTRACTED TEXT

Raw HTML size: 213,053 chars
Extracted text size: 163 chars
Data loss: 99.9%

✓ Found 220 links in raw HTML
✓ Found 79 images in raw HTML

📋 Sample club links from raw HTML:
  : https://www.azsoccerassociation.org/
  About ASA: https://www.azsoccerassociation.org/about-asa/
  Staff: https://www.azsoccerassociation.org/staff/
  Member Clubs: https://www.azsoccerassociation.org/member-clubs/
  Board of Directors: https://www.azsoccerassociation.org/board-of-directors/

📋 Sample image URLs from raw HTML:
  https://www.azsoccerassociation.org/wp-content/uploads/sites/186/2023/07/cropped
  https://www.azsoccerassociation.org/wp-content/uploads/sites/186/2023/07/USYS-1.
  https://www.azsoccerassociation.org/wp-content/uploads/sites/186/2023/09/1.png?w
  https://www.azsoccerassociation.org/wp-content/uploads/sites/186/2023/09/2.png?w
  https://www.azsoccerassociation.org/wp-content/uploads/sites/186/2023/09/3.png?w

❌ Links in extracted text: 0
❌ Images

In [ ]:
import re
from bs4 import BeautifulSoup, NavigableString

def extract_rich_text_from_html(html_content: str) -> str:
    """
    Extract rich text from HTML that preserves important data.
    
    Keeps:
    - Text content (for semantic meaning)
    - Links (href attributes)
    - Images (src, alt attributes)
    - Data attributes (data-*)
    - Important class names
    - Script content (JSON/JavaScript variables)
    
    Removes:
    - CSS styles
    - Navigation/header/footer noise
    - Empty tags
    - Script tags (unless they contain data)
    """
    soup = BeautifulSoup(html_content, 'lxml')
    
    # Remove noise elements
    for element in soup(['style', 'script', 'nav', 'header', 'footer', 'noscript']):
        # Keep scripts that might have JSON data
        if element.name == 'script':
            script_text = element.get_text()
            # Check if it has data (JSON objects, arrays, or variable assignments)
            if any(pattern in script_text for pattern in ['var ', 'const ', 'let ', '{', '[']):
                continue  # Keep this script
        element.decompose()
    
    result_parts = []
    
    def extract_element(element, depth=0):
        """Recursively extract element with its data"""
        
        if isinstance(element, NavigableString):
            text = str(element).strip()
            if text:
                result_parts.append(text)
            return
        
        # Skip if no useful content
        if not element.name:
            return
        
        # Extract links
        if element.name == 'a' and element.get('href'):
            text = element.get_text(strip=True)
            href = element['href']
            if text and href:
                result_parts.append(f"{text} [LINK: {href}]")
            return
        
        # Extract images
        if element.name == 'img':
            src = element.get('src', '')
            alt = element.get('alt', '')
            if src:
                result_parts.append(f"[IMAGE: {src}]")
            if alt:
                result_parts.append(f"[ALT: {alt}]")
            return
        
        # Extract data attributes
        data_attrs = {k: v for k, v in element.attrs.items() if k.startswith('data-')}
        if data_attrs:
            for key, value in data_attrs.items():
                result_parts.append(f"[{key.upper()}: {value}]")
        
        # Extract useful class information (filter out CSS framework classes)
        classes = element.get('class', [])
        useful_classes = [c for c in classes if not any(
            skip in c.lower() for skip in ['css-', 'mui-', 'btn-', 'container', 'row', 'col-']
        )]
        if useful_classes:
            result_parts.append(f"[CLASS: {' '.join(useful_classes)}]")
        
        # Process children
        for child in element.children:
            extract_element(child, depth + 1)
    
    # Extract from body
    body = soup.find('body') or soup
    extract_element(body)
    
    # Extract script data (JSON, variables)
    scripts = soup.find_all('script')
    for script in scripts:
        script_text = script.get_text()
        
        # Try to find JSON objects
        json_pattern = r'\{[^{}]*(?:\{[^{}]*\}[^{}]*)*\}'
        json_matches = re.findall(json_pattern, script_text)
        for match in json_matches[:3]:  # Limit to avoid too much data
            if len(match) < 500:  # Only small objects
                result_parts.append(f"[SCRIPT_DATA: {match}]")
        
        # Try to find variable assignments with coordinates/data
        var_pattern = r'(?:var|const|let)\s+\w+\s*=\s*([^;]+);'
        var_matches = re.findall(var_pattern, script_text)
        for match in var_matches[:5]:
            if any(keyword in match.lower() for keyword in ['lat', 'lng', 'coord', 'location', 'address']):
                result_parts.append(f"[SCRIPT_VAR: {match.strip()}]")
    
    # Join and clean
    rich_text = ' '.join(result_parts)
    
    # Clean up multiple spaces
    rich_text = re.sub(r'\s+', ' ', rich_text)
    
    return rich_text.strip()

print("✓ Smart HTML extraction function defined!")


✓ Smart HTML extraction function defined!


### Debug: Test Smart Extraction on Scenario 3


In [66]:
# Test on scenario 3 (clubs)
print("="*80)
print("SCENARIO 3: Smart Extraction Test")
print("="*80)

with open(HTML_FILES["scenario3_clubs"], 'r') as f:
    html = f.read()

rich_text = extract_rich_text_from_html(html)

print(f"\nOriginal HTML size: {len(html):,} chars")
print(f"Rich text size: {len(rich_text):,} chars")
print(f"Compression: {(1 - len(rich_text)/len(html))*100:.1f}% smaller")

print(f"\n{'='*80}")
print("First 2000 characters of extracted rich text:")
print(f"{'='*80}\n")
print(rich_text[:2000])
print("...")

print(f"\n{'='*80}")
print("Last 2000 characters of extracted rich text:")
print(f"{'='*80}\n")
print(rich_text[-2000:])

# Count data elements
link_count = rich_text.count('[LINK:')
image_count = rich_text.count('[IMAGE:')
data_attr_count = rich_text.count('[DATA-')
script_data_count = rich_text.count('[SCRIPT_')

print(f"\n{'='*80}")
print("📊 Extracted Data Summary:")
print(f"{'='*80}")
print(f"  Links: {link_count}")
print(f"  Images: {image_count}")
print(f"  Data attributes: {data_attr_count}")
print(f"  Script data: {script_data_count}")
print(f"  Total chars: {len(rich_text):,}")
print(f"{'='*80}")


SCENARIO 3: Smart Extraction Test

Original HTML size: 213,053 chars
Rich text size: 27,403 chars
Compression: 87.1% smaller

First 2000 characters of extracted rich text:

Google Tag Manager (noscript) End Google Tag Manager (noscript) //<![CDATA[ (function(){ var c = document.body.classList; c.remove( 'no-js' ); c.add( 'js' ); })(); //]]> Skip to main content [LINK: #genesis-content] Skip to primary navigation [LINK: #menu-main-menu] [DATA-USL-DROPDOWN-CONTAINER: ] [DATA-USL-DROPDOWN: ] [DATA-USL-DROPDOWN-BUTTON: ] More Items [DATA-USL-DROPDOWN-MENU: ] This is a list of every current member club of the Arizona Soccer Association. To identify a club near you please use the map or dropdown list below. Phoenix Arizona Soccer Academy [LINK: https://www.azsocceracademy.com/] Arizona Storm [LINK: https://www.azstormfc.com/] AYSO United [LINK: https://aysounitedsoccer.com/] AZ Arsenal [LINK: https://www.azarsenalsc.org/] AZFC Select [LINK: https://azfcselect.com/] AZGolden Eagles FC [LINK: 

In [67]:
rich_text

'Google Tag Manager (noscript) End Google Tag Manager (noscript) //<![CDATA[ (function(){ var c = document.body.classList; c.remove( \'no-js\' ); c.add( \'js\' ); })(); //]]> Skip to main content [LINK: #genesis-content] Skip to primary navigation [LINK: #menu-main-menu] [DATA-USL-DROPDOWN-CONTAINER: ] [DATA-USL-DROPDOWN: ] [DATA-USL-DROPDOWN-BUTTON: ] More Items [DATA-USL-DROPDOWN-MENU: ] This is a list of every current member club of the Arizona Soccer Association. To identify a club near you please use the map or dropdown list below. Phoenix Arizona Soccer Academy [LINK: https://www.azsocceracademy.com/] Arizona Storm [LINK: https://www.azstormfc.com/] AYSO United [LINK: https://aysounitedsoccer.com/] AZ Arsenal [LINK: https://www.azarsenalsc.org/] AZFC Select [LINK: https://azfcselect.com/] AZGolden Eagles FC [LINK: https://www.azgesoccerafc.org/] Arizona Soccer Club [LINK: https://www.arizonasoccerclub.com/] AZ Inferno [LINK: https://www.azinfernosoccerclub.net/] Brazas Futebol Cl

### Debug: Test Smart Extraction on Scenario 4


In [68]:
# Test on scenario 4 (property)
print("="*80)
print("SCENARIO 4: Smart Extraction Test")
print("="*80)

with open(HTML_FILES["scenario4_property"], 'r') as f:
    html = f.read()

rich_text = extract_rich_text_from_html(html)

print(f"\nOriginal HTML size: {len(html):,} chars")
print(f"Rich text size: {len(rich_text):,} chars")
print(f"Compression: {(1 - len(rich_text)/len(html))*100:.1f}% smaller")

print(f"\n{'='*80}")
print("First 2000 characters of extracted rich text:")
print(f"{'='*80}\n")
print(rich_text[:2000])
print("...")

# Search for coordinate-related data
print(f"\n{'='*80}")
print("🔍 Searching for coordinates in extracted text:")
print(f"{'='*80}\n")

# Look for patterns
patterns = {
    'latitude': r'lat(?:itude)?[:\s]+(-?\d+\.?\d*)',
    'longitude': r'lon(?:g|gitude)?[:\s]+(-?\d+\.?\d*)',
    'coordinates': r'(-?\d{1,3}\.\d+)[,\s]+(-?\d{1,3}\.\d+)',
    'data-lat': r'data-lat[^:]*:\s*([^,\]]+)',
    'script_coords': r'\{\s*["\']?lat["\']?\s*:\s*([^,}]+)',
}

found_coords = {}
for name, pattern in patterns.items():
    matches = re.findall(pattern, rich_text.lower())
    if matches:
        found_coords[name] = matches[:3]  # First 3 matches
        print(f"  ✓ Found {name}: {matches[:3]}")

if not found_coords:
    print("  ❌ No coordinates found")
    print(f"\n  Checking for keywords:")
    for keyword in ['lat', 'lng', 'coord', 'location', 'address', 'property']:
        count = rich_text.lower().count(keyword)
        if count > 0:
            print(f"    '{keyword}': {count} occurrences")

# Count data elements
link_count = rich_text.count('[LINK:')
image_count = rich_text.count('[IMAGE:')
data_attr_count = rich_text.count('[DATA-')
script_data_count = rich_text.count('[SCRIPT_')

print(f"\n{'='*80}")
print("📊 Extracted Data Summary:")
print(f"{'='*80}")
print(f"  Links: {link_count}")
print(f"  Images: {image_count}")
print(f"  Data attributes: {data_attr_count}")
print(f"  Script data: {script_data_count}")
print(f"  Total chars: {len(rich_text):,}")
print(f"{'='*80}")


SCENARIO 4: Smart Extraction Test

Original HTML size: 255,548 chars
Rich text size: 76,439 chars
Compression: 70.1% smaller

First 2000 characters of extracted rich text:

[DATA-IMGLITE-ID: imglite_p2xv0o72q99] [DATA-IMGLITE-VISIBLE: false] [DATA-IMGLITE-ID: imglite_x3a912749lo] [DATA-IMGLITE-VISIBLE: false] [DATA-IMGLITE-ID: imglite_ztlikm0y5gn] [DATA-IMGLITE-VISIBLE: false] [DATA-IMGLITE-ID: imglite_o6ae2nzj0g] [DATA-IMGLITE-VISIBLE: false] [DATA-IMGLITE-ID: imglite_kwiu4wf0z1] [DATA-IMGLITE-VISIBLE: false] [DATA-IMGLITE-ID: imglite_szz6st0jfnf] [DATA-IMGLITE-VISIBLE: false] [DATA-IMGLITE-ID: imglite_5htoju71d1] [DATA-IMGLITE-VISIBLE: false] [DATA-IMGLITE-ID: imglite_coh9pjrr0t7] [DATA-IMGLITE-VISIBLE: false] [DATA-IMGLITE-ID: imglite_v8ubjheolva] [DATA-IMGLITE-VISIBLE: false] [DATA-IMGLITE-ID: imglite_vz6dl7l4u1j] [DATA-IMGLITE-VISIBLE: false] [DATA-IMGLITE-ID: imglite_xjgmjolklf] [DATA-IMGLITE-VISIBLE: false] [DATA-IMGLITE-ID: imglite_u7z2akiyjxh] [DATA-IMGLITE-VISIBLE: false] [DA

### Compare: UnstructuredHTMLLoader vs Smart Extraction


## 13. Integrated RAG with Smart HTML Extraction

Now let's rebuild the RAG system using our smart extraction function


In [69]:
def load_and_index_html_smart(html_path: Path, chunk_size: int = 1000, overlap: int = 200):
    """
    Load HTML using SMART extraction and create vector index.
    
    This version:
    1. Extracts rich text (preserving links, images, data attributes, scripts)
    2. Chunks the rich text
    3. Creates embeddings
    4. Indexes for semantic search
    """
    print(f"  Loading and indexing {html_path.name} with smart extraction...")
    
    # Load raw HTML
    with open(html_path, 'r', encoding='utf-8') as f:
        html_content = f.read()
    
    # Extract rich text (preserves data!)
    rich_text = extract_rich_text_from_html(html_content)
    
    print(f"    HTML: {len(html_content):,} chars → Rich text: {len(rich_text):,} chars")
    
    # Create document
    doc = Document(page_content=rich_text, metadata={"source": str(html_path)})
    
    # Chunk the rich text
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=overlap
    )
    chunks = splitter.split_documents([doc])
    
    # Create vector store
    vectorstore = Chroma.from_documents(
        chunks,
        embeddings,
        collection_name=f"smart_{html_path.stem}"
    )
    
    print(f"    ✓ Indexed {len(chunks)} chunks")
    return vectorstore

print("✓ Smart RAG loader defined!")


✓ Smart RAG loader defined!


In [70]:
def iterative_rag_extraction_smart(
    html_path: Path,
    query: str,
    max_iterations: int = 3,
    initial_k: int = 20,
    max_k: int = 50
) -> List[Dict]:
    """
    Iterative RAG extraction with SMART HTML parsing.
    
    Key difference: Uses extract_rich_text_from_html instead of UnstructuredHTMLLoader
    """
    print(f"\n{'='*80}")
    print(f"Smart Iterative RAG: {html_path.name}")
    print(f"Query: {query}")
    print(f"{'='*80}\n")
    
    # Load and index with smart extraction
    vectorstore = load_and_index_html_smart(html_path, chunk_size=800, overlap=100)
    print()
    
    all_results = []
    k = initial_k
    
    # Query variations
    query_variations = [
        query,
        f"all {query}",
        f"complete list {query}",
        query.replace("extract", "find all").replace("get", "list all")
    ]
    
    for iteration in range(max_iterations):
        print(f"[Iteration {iteration + 1}]")
        
        # Use different query each iteration
        current_query = query_variations[min(iteration, len(query_variations) - 1)]
        
        # Retrieve more chunks each iteration
        current_k = min(k + (iteration * 15), max_k)
        print(f"  Retrieving top {current_k} chunks for: '{current_query[:50]}...'")
        
        chunks = search_and_retrieve(vectorstore, current_query, k=current_k)
        print(f"  Retrieved {len(chunks)} chunks ({sum(len(c) for c in chunks):,} chars)")
        
        # Extract
        print(f"  Extracting data from chunks...")
        results = extract_from_chunks(chunks, query, llm)
        print(f"  ✓ Extracted {len(results)} items")
        
        all_results.extend(results)
        
        # Check progress
        unique_so_far = deduplicate_results(all_results)
        print(f"  Total unique items so far: {len(unique_so_far)}\n")
        
        # Stop if no new results
        if iteration > 0 and len(unique_so_far) == len(deduplicate_results(all_results[:-len(results)])):
            print(f"  No new items found, stopping.\n")
            break
    
    # Final deduplication
    final_results = deduplicate_results(all_results)
    
    print(f"{'='*80}")
    print(f"✓ FINAL: {len(final_results)} unique items")
    print(f"{'='*80}\n")
    
    return final_results

print("✓ Smart iterative RAG function defined!")


✓ Smart iterative RAG function defined!


## 14. Test Smart RAG on All Scenarios


In [71]:
# Run smart RAG on all scenarios
smart_rag_results = {}

for scenario_name, html_file in HTML_FILES.items():
    query = TEST_QUERIES[scenario_name]
    
    try:
        result = iterative_rag_extraction_smart(html_file, query, max_iterations=1, initial_k=20)
        smart_rag_results[scenario_name] = result
        
        print("\n" + "="*80)
        print(f"RESULT for {scenario_name}:")
        print(f"✓ Extracted {len(result)} items\n")
        
        if len(result) > 0:
            print("First 2 items:")
            print(json.dumps(result[:2], indent=2))
        
        print("="*80 + "\n\n")
        
    except Exception as e:
        print(f"\n✗ Error: {e}\n")
        import traceback
        traceback.print_exc()
        smart_rag_results[scenario_name] = []



Smart Iterative RAG: scenario1_books.html
Query: Can you return me the books: name and price?

  Loading and indexing scenario1_books.html with smart extraction...
    HTML: 51,274 chars → Rich text: 6,902 chars
    ✓ Indexed 10 chunks

[Iteration 1]
  Retrieving top 20 chunks for: 'Can you return me the books: name and price?...'
  Retrieved 20 chunks (18,625 chars)
  Extracting data from chunks...
  ✓ Extracted 3 items
  Total unique items so far: 3

✓ FINAL: 3 unique items


RESULT for scenario1_books:
✓ Extracted 3 items

First 2 items:
[
  {
    "name": "Historical Fiction",
    "price": null
  },
  {
    "name": "Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991",
    "price": 57.25
  }
]



Smart Iterative RAG: scenario2_jobs.html
Query: Extract job title, location, salary, and company name from the listings

  Loading and indexing scenario2_jobs.html with smart extraction...
    HTML: 497,323 chars → Rich text: 119,761 chars
    ✓ Indexed 172 c

## 15. Final Comparison: All Approaches


In [34]:
print("\n" + "="*80)
print("FINAL COMPARISON: All Approaches")
print("="*80 + "\n")

# Historical results from previous experiments
comparison = {
    "Original RAG (k=5, UnstructuredLoader)": {
        "scenario1_books": 20,
        "scenario2_jobs": 7,
        "scenario3_clubs": 2,
        "scenario4_property": 0
    },
    "Code Generation": {
        "scenario1_books": 0,
        "scenario2_jobs": 0,
        "scenario3_clubs": 6,
        "scenario4_property": 0
    },
    "Iterative RAG (k=20-50, UnstructuredLoader)": {
        "scenario1_books": "?",
        "scenario2_jobs": "?",
        "scenario3_clubs": "?",
        "scenario4_property": "?"
    },
    "Smart RAG (k=20-50, Smart Extraction)": {
        scenario: len(result) for scenario, result in smart_rag_results.items()
    }
}

# Print table
print(f"{'Scenario':<25} {'Original':<12} {'Code Gen':<12} {'Iter RAG':<12} {'Smart RAG':<12}")
print("-" * 80)

for scenario in HTML_FILES.keys():
    orig = comparison["Original RAG (k=5, UnstructuredLoader)"][scenario]
    code = comparison["Code Generation"][scenario]
    iter_rag = comparison["Iterative RAG (k=20-50, UnstructuredLoader)"][scenario]
    smart = comparison["Smart RAG (k=20-50, Smart Extraction)"][scenario]
    
    print(f"{scenario:<25} {str(orig):<12} {str(code):<12} {str(iter_rag):<12} {str(smart):<12}")

print("\n" + "="*80)
print("\n🎯 KEY INSIGHTS:\n")
print("1. UnstructuredHTMLLoader loses 99.9% of data (links, images, attributes)")
print("2. Smart extraction preserves all data while removing noise")
print("3. Code generation hallucinates selectors")
print("4. Smart RAG = Iterative retrieval + Rich data = Complete extraction")
print("\n" + "="*80)



FINAL COMPARISON: All Approaches

Scenario                  Original     Code Gen     Iter RAG     Smart RAG   
--------------------------------------------------------------------------------
scenario1_books           20           0            ?            23          
scenario2_jobs            7            0            ?            11          
scenario3_clubs           2            6            ?            35          
scenario4_property        0            0            ?            26          


🎯 KEY INSIGHTS:

1. UnstructuredHTMLLoader loses 99.9% of data (links, images, attributes)
2. Smart extraction preserves all data while removing noise
3. Code generation hallucinates selectors
4. Smart RAG = Iterative retrieval + Rich data = Complete extraction



## 16. The Winning Formula

### What We Discovered

**Problem 1**: Original RAG incomplete
- **Root cause**: k=5 too small
- **Solution**: Increase k=20-50 ✅

**Problem 2**: Code generation hallucinated
- **Root cause**: LLM guessing HTML structure
- **Solution**: Skip code generation, extract directly ✅

**Problem 3**: Still failing on scenarios 3 & 4
- **Root cause**: UnstructuredHTMLLoader strips 99.9% of data!
- **Solution**: Smart extraction preserves links/images/data attributes ✅

### The Complete Solution

```python
# Smart Extraction (preserves data)
rich_text = extract_rich_text_from_html(html)
# Output: "FC Sonora [LINK: https://fcsonora.com] [IMAGE: logo.png]"

# Iterative Retrieval (completeness)
for iteration in range(3):
    k = 20 + (iteration * 15)  # Progressive: 20 → 35 → 50
    chunks = search(query, k=k)
    results.extend(extract(chunks))

# Direct Extraction (no hallucination)
llm.extract_from_text(chunks)  # Not code generation!
```

### Why This Works

| Component | Purpose | Benefit |
|-----------|---------|---------|
| Smart Extraction | Preserve data in HTML | Links, images, coords available |
| Embeddings | Find relevant sections | Semantic search works |
| High K | Get more context | Completeness |
| Iterations | Progressive refinement | Catch edge cases |
| Direct Extraction | LLM reads provided text | No hallucination |

This is the **universal solution** for HTML extraction! 🎯


## Fix: Clear Chroma Cache (for new embedding model)


In [64]:
# Clear Chroma cache to fix dimension mismatch
import shutil
import tempfile

# Chroma stores data in temp directory by default
chroma_dir = Path(tempfile.gettempdir()) / "chroma"
if chroma_dir.exists():
    try:
        shutil.rmtree(chroma_dir)
        print(f"✓ Cleared Chroma cache at {chroma_dir}")
    except Exception as e:
        print(f"⚠️  Could not clear cache: {e}")
else:
    print("ℹ️  No Chroma cache found")

# Also check current directory
local_chroma = Path("./chroma")
if local_chroma.exists():
    try:
        shutil.rmtree(local_chroma)
        print(f"✓ Cleared local Chroma cache")
    except Exception as e:
        print(f"⚠️  Could not clear local cache: {e}")

print("\n✓ Ready to use new embedding model!")


ℹ️  No Chroma cache found

✓ Ready to use new embedding model!


## Alternative Fix: Use Unique Collection Names


In [6]:
def extract_rich_text_from_html_enhanced(html_content: str) -> str:
    """
    Enhanced HTML extraction that preserves:
    1. All useful element types (li, p, div, span, etc.)
    2. All useful attributes (id, title, aria-*, name, value, etc.)
    3. Structural information (lists, tables, sections)
    """
    from bs4 import BeautifulSoup, NavigableString
    import re
    
    soup = BeautifulSoup(html_content, 'lxml')
    
    # Remove noise elements
    for element in soup(['style', 'script', 'nav', 'header', 'footer', 'noscript']):
        if element.name == 'script':
            script_text = element.get_text()
            if any(pattern in script_text for pattern in ['var ', 'const ', 'let ', '{', '[']):
                continue
        element.decompose()
    
    result_parts = []
    
    # Attributes to always extract (contain useful data)
    USEFUL_ATTRS = [
        'id', 'name', 'value', 'title', 'alt', 'src', 'href',
        'aria-label', 'aria-describedby', 'placeholder', 'type',
        'role', 'rel', 'target'
    ]
    
    # Element types that indicate structure
    STRUCTURAL_ELEMENTS = {
        'ul': 'LIST', 'ol': 'ORDERED_LIST', 'li': 'ITEM',
        'table': 'TABLE', 'tr': 'ROW', 'td': 'CELL', 'th': 'HEADER',
        'article': 'ARTICLE', 'section': 'SECTION',
        'figure': 'FIGURE', 'figcaption': 'CAPTION'
    }
    
    def extract_element(element, depth=0):
        """Recursively extract element with ALL useful data"""
        
        if isinstance(element, NavigableString):
            text = str(element).strip()
            if text:
                result_parts.append(text)
            return
        
        if not element.name:
            return
        
        # Mark structural elements
        if element.name in STRUCTURAL_ELEMENTS:
            result_parts.append(f"[{STRUCTURAL_ELEMENTS[element.name]}]")
        
        # Extract links with full context
        if element.name == 'a':
            text = element.get_text(strip=True)
            href = element.get('href', '')
            title = element.get('title', '')
            
            if text or href:
                link_info = f"{text}" if text else ""
                if href:
                    link_info += f" [LINK: {href}]"
                if title:
                    link_info += f" [TITLE: {title}]"
                result_parts.append(link_info)
            
            # Don't process children (already got text)
            return
        
        # Extract images with all metadata
        if element.name == 'img':
            src = element.get('src', '')
            alt = element.get('alt', '')
            title = element.get('title', '')
            
            if src:
                result_parts.append(f"[IMAGE: {src}]")
            if alt:
                result_parts.append(f"[ALT: {alt}]")
            if title:
                result_parts.append(f"[IMG_TITLE: {title}]")
            return
        
        # Extract form inputs (contain data!)
        if element.name in ['input', 'select', 'textarea', 'button']:
            attrs_to_extract = {}
            for attr in ['name', 'value', 'placeholder', 'type', 'id']:
                if element.get(attr):
                    attrs_to_extract[attr] = element.get(attr)
            
            if attrs_to_extract:
                attrs_str = ' '.join([f"{k}=\"{v}\"" for k, v in attrs_to_extract.items()])
                result_parts.append(f"[{element.name.upper()}: {attrs_str}]")
        
        # Extract ALL data attributes
        data_attrs = {k: v for k, v in element.attrs.items() if k.startswith('data-')}
        if data_attrs:
            for key, value in data_attrs.items():
                result_parts.append(f"[{key.upper()}: {value}]")
        
        # Extract other useful attributes
        for attr in USEFUL_ATTRS:
            if attr not in ['href', 'src', 'alt']:  # Already handled above
                value = element.get(attr)
                if value and attr not in ['class']:  # Skip class for now
                    result_parts.append(f"[{attr.upper()}: {value}]")
        
        # Extract useful class information
        classes = element.get('class', [])
        useful_classes = [c for c in classes if not any(
            skip in c.lower() for skip in ['css-', 'mui-', 'btn-', 'container', 'row', 'col-', 'fa-', 'icon-']
        )]
        if useful_classes and len(useful_classes) <= 3:  # Limit to avoid noise
            result_parts.append(f"[CLASS: {' '.join(useful_classes)}]")
        
        # Process children recursively
        for child in element.children:
            extract_element(child, depth + 1)
        
        # Mark end of structural elements
        if element.name in STRUCTURAL_ELEMENTS and element.name in ['ul', 'ol', 'table']:
            result_parts.append(f"[/{STRUCTURAL_ELEMENTS[element.name]}]")
    
    # Extract from body
    body = soup.find('body') or soup
    extract_element(body)
    
    # Extract script data (same as before)
    scripts = soup.find_all('script')
    for script in scripts:
        script_text = script.get_text()
        
        # JSON objects
        json_pattern = r'\{[^{}]*(?:\{[^{}]*\}[^{}]*)*\}'
        json_matches = re.findall(json_pattern, script_text)
        for match in json_matches[:3]:
            if len(match) < 500:
                result_parts.append(f"[SCRIPT_DATA: {match}]")
        
        # Variable assignments
        var_pattern = r'(?:var|const|let)\s+\w+\s*=\s*([^;]+);'
        var_matches = re.findall(var_pattern, script_text)
        for match in var_matches[:5]:
            if any(keyword in match.lower() for keyword in ['lat', 'lng', 'coord', 'location', 'address']):
                result_parts.append(f"[SCRIPT_VAR: {match.strip()}]")
    
    # Join and clean
    rich_text = ' '.join(result_parts)
    rich_text = re.sub(r'\s+', ' ', rich_text)
    
    return rich_text.strip()

print("✓ Enhanced HTML extraction function defined!")


✓ Enhanced HTML extraction function defined!


## Test Enhanced HTML Extraction

Let's test the enhanced extraction on all scenarios to see the improvement


In [8]:
# Test enhanced extraction on all scenarios
print("="*80)
print("TESTING ENHANCED HTML EXTRACTION")
print("="*80 + "\n")

for scenario_name, html_file in HTML_FILES.items():
    print(f"\n{'-'*80}")
    print(f"SCENARIO: {scenario_name}")
    print(f"{'-'*80}")
    
    # Load raw HTML
    with open(html_file, 'r', encoding='utf-8') as f:
        html_content = f.read()
    
    # Extract with enhanced function
    rich_text = extract_rich_text_from_html_enhanced(html_content)
    
    # Stats
    print(f"  Original HTML: {len(html_content):,} chars")
    print(f"  Enhanced text: {len(rich_text):,} chars")
    print(f"  Compression: {(1 - len(rich_text)/len(html_content))*100:.1f}%")
    
    # Count extracted elements
    link_count = rich_text.count('[LINK:')
    image_count = rich_text.count('[IMAGE:')
    data_attr_count = rich_text.count('[DATA-')
    list_count = rich_text.count('[LIST]')
    item_count = rich_text.count('[ITEM]')
    
    print(f"\n  📊 Extracted Elements:")
    print(f"     Links: {link_count}")
    print(f"     Images: {image_count}")
    print(f"     Data attributes: {data_attr_count}")
    print(f"     Lists: {list_count}")
    print(f"     List items: {item_count}")
    
    # Show sample of extracted text
    print(f"\n  📝 Sample (first 500 chars):")
    print(f"     {rich_text[:10000]}")
    if len(rich_text) > 10000:
        print(f"     ...")
    
    print()

print("\n" + "="*80)
print("✓ Enhanced extraction preserves structure and data!")
print("="*80)


TESTING ENHANCED HTML EXTRACTION


--------------------------------------------------------------------------------
SCENARIO: scenario1_books
--------------------------------------------------------------------------------
  Original HTML: 51,274 chars
  Enhanced text: 13,845 chars
  Compression: 73.0%

  📊 Extracted Elements:
     Links: 93
     Images: 0
     Data attributes: 20
     Lists: 4
     List items: 75

  📝 Sample (first 500 chars):
     [ID: default] [CLASS: default] [CLASS: page] [CLASS: page_inner] [LIST] [CLASS: breadcrumb] [ITEM] Home [LINK: index.html] [ITEM] [CLASS: active] All products [/LIST] [CLASS: sidebar] [ID: promotions_left] [CLASS: side_categories] [LIST] [CLASS: nav nav-list] [ITEM] Books [LINK: catalogue/category/books_1/index.html] [LIST] [ITEM] Travel [LINK: catalogue/category/books/travel_2/index.html] [ITEM] Mystery [LINK: catalogue/category/books/mystery_3/index.html] [ITEM] Historical Fiction [LINK: catalogue/category/books/historical-fiction_4/index

In [75]:
import time

def load_and_index_html_smart_v2(html_path: Path, chunk_size: int = 1000, overlap: int = 200):
    """
    Load HTML using SMART extraction and create vector index.
    Uses unique collection name to avoid dimension mismatch.
    """
    print(f"  Loading and indexing {html_path.name} with smart extraction...")
    
    # Load raw HTML
    with open(html_path, 'r', encoding='utf-8') as f:
        html_content = f.read()
    
    # Extract rich text (preserves data!)
    rich_text = extract_rich_text_from_html_enhanced(html_content)
    
    print(f"    HTML: {len(html_content):,} chars → Rich text: {len(rich_text):,} chars")
    
    # Create document
    doc = Document(page_content=rich_text, metadata={"source": str(html_path)})
    
    # Chunk the rich text
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=overlap
    )
    chunks = splitter.split_documents([doc])
    
    # Create vector store with UNIQUE collection name (avoids dimension mismatch)
    collection_name = f"smart_{html_path.stem}_{int(time.time())}"
    vectorstore = Chroma.from_documents(
        chunks,
        embeddings,
        collection_name=collection_name
    )
    
    print(f"    ✓ Indexed {len(chunks)} chunks")
    return vectorstore

# Update the main function to use v2
def iterative_rag_extraction_smart_v2(
    html_path: Path,
    query: str,
    max_iterations: int = 3,
    initial_k: int = 20,
    max_k: int = 50
) -> List[Dict]:
    """
    Iterative RAG extraction with SMART HTML parsing (v2 - fixed dimension issue)
    """
    print(f"\n{'='*80}")
    print(f"Smart Iterative RAG v2: {html_path.name}")
    print(f"Query: {query}")
    print(f"{'='*80}\n")
    
    # Load and index with smart extraction (v2)
    vectorstore = load_and_index_html_smart_v2(html_path, chunk_size=1000, overlap=200)
    print()
    
    all_results = []
    k = initial_k
    
    # Query variations
    query_variations = [
        query,
        f"all {query}",
        f"complete list {query}",
        query.replace("extract", "find all").replace("get", "list all")
    ]
    
    for iteration in range(max_iterations):
        print(f"[Iteration {iteration + 1}]")
        
        # Use different query each iteration
        current_query = query_variations[min(iteration, len(query_variations) - 1)]
        
        # Retrieve more chunks each iteration
        current_k = min(k + (iteration * 15), max_k)
        print(f"  Retrieving top {current_k} chunks for: '{current_query[:50]}...'")
        
        chunks = search_and_retrieve(vectorstore, current_query, k=current_k)
        print(f"  Retrieved {len(chunks)} chunks ({sum(len(c) for c in chunks):,} chars)")
        
        # Extract
        print(f"  Extracting data from chunks...")
        results = extract_from_chunks(chunks, query, llm)
        print(f"  ✓ Extracted {len(results)} items")
        
        all_results.extend(results)
        
        # Check progress
        unique_so_far = deduplicate_results(all_results)
        print(f"  Total unique items so far: {len(unique_so_far)}\n")
        
        # Stop if no new results
        if iteration > 0 and len(unique_so_far) == len(deduplicate_results(all_results[:-len(results)])):
            print(f"  No new items found, stopping.\n")
            break
    
    # Final deduplication
    final_results = deduplicate_results(all_results)
    
    print(f"{'='*80}")
    print(f"✓ FINAL: {len(final_results)} unique items")
    print(f"{'='*80}\n")
    
    return final_results

print("✓ Fixed version (v2) defined!")


✓ Fixed version (v2) defined!


## Re-run with Fixed Version


In [76]:
# Run smart RAG v2 on all scenarios
smart_rag_results_v2 = {}

for scenario_name, html_file in HTML_FILES.items():
    query = TEST_QUERIES[scenario_name]
    
    try:
        result = iterative_rag_extraction_smart_v2(html_file, query, max_iterations=1, initial_k=20)
        smart_rag_results_v2[scenario_name] = result
        
        print("\n" + "="*80)
        print(f"RESULT for {scenario_name}:")
        print(f"✓ Extracted {len(result)} items\n")
        
        if len(result) > 0:
            print("First 2 items:")
            print(json.dumps(result[:2], indent=2))
        
        print("="*80 + "\n\n")
        
    except Exception as e:
        print(f"\n✗ Error: {e}\n")
        import traceback
        traceback.print_exc()
        smart_rag_results_v2[scenario_name] = []



Smart Iterative RAG v2: scenario1_books.html
Query: Can you return me the books: name and price?

  Loading and indexing scenario1_books.html with smart extraction...
    HTML: 51,274 chars → Rich text: 13,845 chars
    ✓ Indexed 18 chunks

[Iteration 1]
  Retrieving top 20 chunks for: 'Can you return me the books: name and price?...'
  Retrieved 18 chunks (17,009 chars)
  Extracting data from chunks...
  ✓ Extracted 24 items
  Total unique items so far: 24

✓ FINAL: 24 unique items


RESULT for scenario1_books:
✓ Extracted 24 items

First 2 items:
[
  {
    "name": "A Light in the Attic",
    "price": 51.77
  },
  {
    "name": "Tipping the Velvet",
    "price": 53.74
  }
]



Smart Iterative RAG v2: scenario2_jobs.html
Query: Extract job title, location, salary, and company name from the listings

  Loading and indexing scenario2_jobs.html with smart extraction...
    HTML: 497,323 chars → Rich text: 143,417 chars
    ✓ Indexed 179 chunks

[Iteration 1]
  Retrieving top 20 chunks fo

## V3: Sliding Window Approach

Instead of looking at top K (overlapping), look at DIFFERENT windows each iteration


In [77]:
def search_and_retrieve_window(vectorstore, query: str, offset: int = 0, window_size: int = 20) -> List[str]:
    """
    Retrieve a WINDOW of chunks starting at offset.
    
    This allows us to explore different parts of the document each iteration.
    """
    # Retrieve more than we need, then slice
    total_to_fetch = offset + window_size
    docs = vectorstore.similarity_search(query, k=total_to_fetch)
    
    # Return only the window we want
    window_docs = docs[offset:offset + window_size]
    
    return [doc.page_content for doc in window_docs]

def iterative_rag_extraction_sliding_window(
    html_path: Path,
    query: str,
    max_iterations: int = 5,
    window_size: int = 30,  # How many chunks per iteration
    max_total_chunks: int = 150  # Stop after examining this many chunks total
) -> List[Dict]:
    """
    Iterative RAG with SLIDING WINDOW.
    
    Instead of:
      Iter 1: Top 20 chunks
      Iter 2: Top 35 chunks (same 20 + 15 new)
    
    We do:
      Iter 1: Chunks 0-30
      Iter 2: Chunks 30-60 (completely different!)
      Iter 3: Chunks 60-90 (completely different!)
    
    This explores MORE of the document space.
    """
    print(f"\n{'='*80}")
    print(f"Smart Iterative RAG v3 (Sliding Window): {html_path.name}")
    print(f"Query: {query}")
    print(f"Window size: {window_size} chunks per iteration")
    print(f"{'='*80}\n")
    
    # Load and index with smart extraction
    vectorstore = load_and_index_html_smart_v2(html_path, chunk_size=800, overlap=100)
    print()
    
    all_results = []
    chunks_examined = 0
    
    # Query variations
    query_variations = [
        query,
        f"all {query}",
        f"complete list {query}",
        query.replace("extract", "find all").replace("get", "list all")
    ]
    
    for iteration in range(max_iterations):
        # Stop if we've examined enough chunks
        if chunks_examined >= max_total_chunks:
            print(f"  Reached max chunks limit ({max_total_chunks}), stopping.\n")
            break
        
        print(f"[Iteration {iteration + 1}]")
        
        # Use different query each iteration
        current_query = query_variations[min(iteration, len(query_variations) - 1)]
        
        # Calculate window offset (sliding window!)
        offset = iteration * window_size
        
        print(f"  Retrieving chunks {offset}-{offset + window_size} for: '{current_query[:50]}...'")
        
        # Retrieve THIS window (not overlapping with previous!)
        try:
            chunks = search_and_retrieve_window(vectorstore, current_query, offset=offset, window_size=window_size)
            
            if not chunks:
                print(f"  No more chunks available, stopping.\n")
                break
            
            print(f"  Retrieved {len(chunks)} chunks ({sum(len(c) for c in chunks):,} chars)")
            chunks_examined += len(chunks)
            
        except Exception as e:
            print(f"  Could not retrieve more chunks: {e}")
            break
        
        # Extract
        print(f"  Extracting data from chunks...")
        results = extract_from_chunks(chunks, query, llm)
        print(f"  ✓ Extracted {len(results)} items")
        
        all_results.extend(results)
        
        # Check progress
        unique_so_far = deduplicate_results(all_results)
        print(f"  Total unique items so far: {len(unique_so_far)}")
        print(f"  Total chunks examined: {chunks_examined}\n")
        
        # Stop if no new results in last 2 iterations
        if iteration > 1 and len(unique_so_far) == len(deduplicate_results(all_results[:-len(results)])):
            print(f"  No new items found, stopping.\n")
            break
    
    # Final deduplication
    final_results = deduplicate_results(all_results)
    
    print(f"{'='*80}")
    print(f"✓ FINAL: {len(final_results)} unique items from {chunks_examined} chunks")
    print(f"{'='*80}\n")
    
    return final_results

print("✓ Sliding window RAG function defined!")


✓ Sliding window RAG function defined!


## Test Sliding Window on All Scenarios


In [80]:
# Run sliding window RAG on all scenarios
sliding_window_results = {}

for scenario_name, html_file in HTML_FILES.items():
    query = TEST_QUERIES[scenario_name]
    
    try:
        result = iterative_rag_extraction_sliding_window(
            html_file, 
            query, 
            max_iterations=5,
            window_size=20,
            max_total_chunks=150
        )
        sliding_window_results[scenario_name] = result
        
        print("\n" + "="*80)
        print(f"RESULT for {scenario_name}:")
        print(f"✓ Extracted {len(result)} items\n")
        
        if len(result) > 0:
            print("First 2 items:")
            print(json.dumps(result[:2], indent=2))
        
        print("="*80 + "\n\n")
        
    except Exception as e:
        print(f"\n✗ Error: {e}\n")
        import traceback
        traceback.print_exc()
        sliding_window_results[scenario_name] = []



Smart Iterative RAG v3 (Sliding Window): scenario1_books.html
Query: Can you return me the books: name and price?
Window size: 20 chunks per iteration

  Loading and indexing scenario1_books.html with smart extraction...
    HTML: 51,274 chars → Rich text: 13,845 chars
    ✓ Indexed 20 chunks

[Iteration 1]
  Retrieving chunks 0-20 for: 'Can you return me the books: name and price?...'
  Retrieved 20 chunks (15,474 chars)
  Extracting data from chunks...
  ✓ Extracted 20 items
  Total unique items so far: 13
  Total chunks examined: 20

[Iteration 2]
  Retrieving chunks 20-40 for: 'all Can you return me the books: name and price?...'
  No more chunks available, stopping.

✓ FINAL: 13 unique items from 20 chunks


RESULT for scenario1_books:
✓ Extracted 13 items

First 2 items:
[
  {
    "name": "A Light in the Attic",
    "price": 51.77
  },
  {
    "name": "Tipping the Velvet",
    "price": 53.74
  }
]



Smart Iterative RAG v3 (Sliding Window): scenario2_jobs.html
Query: Extract job

In [81]:
sliding_window_results

{'scenario1_books': [{'name': 'A Light in the Attic', 'price': 51.77},
  {'name': 'Tipping the Velvet', 'price': 53.74},
  {'name': 'Soumission', 'price': 50.1},
  {'name': "It's Only the Himalayas", 'price': 45.17},
  {'name': 'Libertarianism for Beginners', 'price': 51.33},
  {'name': 'Its Only the Himalayas', 'price': 50.1},
  {'name': 'Sharp Objects', 'price': 47.82},
  {'name': 'Sapiens: A Brief History of Humankind', 'price': 54.23},
  {'name': 'Set Me Free', 'price': 17.46},
  {'name': "Scott Pilgrim's Precious Little Life (Scott Pilgrim #1)",
   'price': None},
  {'name': 'The Black Maria', 'price': 52.15},
  {'name': 'Starving Hearts (Triangular Trade Trilogy, #1)', 'price': None},
  {'name': 'The Coming Woman: A Novel Based on the Life of the Infamous Feminist',
   'price': None}],
 'scenario2_jobs': [{'jobTitle': 'Resident Medical Officer',
   'location': 'North Tamworth, New South Wales, AU',
   'salary': None,
   'companyName': 'Tamworth Hospital'},
  {'jobTitle': 'Residen

## Comparison: Overlapping vs Sliding Window


In [ ]:
print("\n" + "="*80)
print("COMPARISON: Overlapping Window vs Sliding Window")
print("="*80 + "\n")

print(f"{'Scenario':<25} {'V2 (Overlap)':<15} {'V3 (Sliding)':<15} {'Improvement':<15}")
print("-" * 80)

for scenario in HTML_FILES.keys():
    v2_count = len(smart_rag_results_v2.get(scenario, []))
    v3_count = len(sliding_window_results.get(scenario, []))
    
    improvement = ""
    if v3_count > v2_count:
        improvement = f"+{v3_count - v2_count} ✅"
    elif v3_count == v2_count:
        improvement = "same"
    else:
        improvement = f"{v3_count - v2_count}"
    
    print(f"{scenario:<25} {v2_count:<15} {v3_count:<15} {improvement:<15}")

print("\n" + "="*80)
print("\n🎯 KEY INSIGHT:\n")
print("Sliding window explores MORE of the document:")
print("  - V2: Looks at top-K ranked chunks (overlapping)")
print("  - V3: Looks at DIFFERENT windows each iteration")
print("  - Result: Finds items that weren't in top-K ranking!")
print("\n" + "="*80)
